# Capstone Project:The Battle of Neighborhoods (Week 2)

## Table of contents
* [Background](#Background)
* [Problem](#Probulem) 
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1) Background<a name="Background"></a>

I want to go back to my home town Toyota-city.  
However my wife was born in another area in Aichi, she is hesitating to go to Toyota-city.  
To overcome the situation I am planing to satisfy her dream by go back to Toyota-city.  
My wife's dream is to open a resturant.  
So showing the potential of Toyota-city for new resturant in this project.

## 2) Problem<a name="Problem"></a>

The idea of this study is to show which part of Toyota-city is the best for new resturant by providing data about:  
potential customers from population derived from population distribution. 
competitors in the same regions.

## 3) Data<a name="Data"></a>

For the study, population and resturant data in accordance with geographic information are needed.  
We can download population vs area name of Toyota-city from the URL below.

https://www.city.toyota.aichi.jp/_res/projects/default_project/_page_/001/004/759/h29/01.xls

Also we can download postcode list from the URL below.

https://www.post.japanpost.jp/zipcode/dl/roman-zip.html

Above two data can be linked by using area name and can be used to obtain geographic information(latitude & longitude) vs population.
In paticular, I used below API to get geographic information from postcode.

'http://geoapi.heartrails.com/api/xml?method=searchByPostal&postal='

Also we can get the resturant information by Foursquare API.
This resulting in linked data among geographic information, population and competiters for new resturant. 


### Post code and area name of Toyota-city

In [1]:
import pandas as pd
import numpy as np
import urllib.request, urllib.error
import xml.etree.ElementTree
import requests

In [2]:
df = pd.read_csv("KEN_ALL_ROME.CSV", encoding='cp932')
df.head(10)

,Postcode,PrefJP,BoroughJP,AreaJP,Pref,Borough,Area
0,600000,北海道,札幌市 中央区,以下に掲載がない場合,HOKKAIDO,SAPPORO SHI CHUO KU,IKANIKEISAIGANAIBAAI
1,640941,北海道,札幌市 中央区,旭ケ丘,HOKKAIDO,SAPPORO SHI CHUO KU,ASAHIGAOKA
2,600041,北海道,札幌市 中央区,大通東,HOKKAIDO,SAPPORO SHI CHUO KU,ODORIHIGASHI
3,600042,北海道,札幌市 中央区,大通西（１～１９丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,ODORINISHI(1-19-CHOME)
4,640820,北海道,札幌市 中央区,大通西（２０～２８丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,ODORINISHI(20-28-CHOME)
5,600031,北海道,札幌市 中央区,北一条東,HOKKAIDO,SAPPORO SHI CHUO KU,KITA1-JOHIGASHI
6,600001,北海道,札幌市 中央区,北一条西（１～１９丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,KITA1-JONISHI(1-19-CHOME)
7,640821,北海道,札幌市 中央区,北一条西（２０～２８丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,KITA1-JONISHI(20-28-CHOME)
8,600032,北海道,札幌市 中央区,北二条東,HOKKAIDO,SAPPORO SHI CHUO KU,KITA2-JOHIGASHI
9,600002,北海道,札幌市 中央区,北二条西（１～１９丁目）,HOKKAIDO,SAPPORO SHI CHUO KU,KITA2-JONISHI(1-19-CHOME)


In [3]:
df = df[df['Borough'] == 'TOYOTA SHI'].reset_index()
df = df[['Postcode', 'AreaJP', 'Pref', 'Borough', 'Area']]
df.AreaJP.astype(str)
df.head()

,Postcode,AreaJP,Pref,Borough,Area
0,4710000,以下に掲載がない場合,AICHI KEN,TOYOTA SHI,IKANIKEISAIGANAIBAAI
1,4710052,逢妻町,AICHI KEN,TOYOTA SHI,AIZUMACHO
2,4700335,青木町,AICHI KEN,TOYOTA SHI,AOKICHO
3,4710873,秋葉町,AICHI KEN,TOYOTA SHI,AKIBACHO
4,4710835,曙町,AICHI KEN,TOYOTA SHI,AKEBONOCHO


### Area name and population of Toyota-city

In [4]:
df_pop = pd.read_csv("01.csv", encoding='cp932')
#df_pop.AreaJP.astype(str)
df_pop.head()

,AreaJP,Population
0,逢妻町,684
1,秋葉町,"1,906"
2,曙町,"1,055"
3,朝日ケ丘,"2,560"
4,朝日町,"3,759"


In [5]:
df = pd.merge(df, df_pop,on='AreaJP', how='inner')
df.head()

,Postcode,AreaJP,Pref,Borough,Area,Population
0,4710052,逢妻町,AICHI KEN,TOYOTA SHI,AIZUMACHO,684
1,4700335,青木町,AICHI KEN,TOYOTA SHI,AOKICHO,"4,527"
2,4710873,秋葉町,AICHI KEN,TOYOTA SHI,AKIBACHO,"1,906"
3,4710835,曙町,AICHI KEN,TOYOTA SHI,AKEBONOCHO,"1,055"
4,4442832,浅谷町,AICHI KEN,TOYOTA SHI,AZAKAICHO,157


### Geographical data download

In [6]:
lat_long = []

def geo(zip_code, API_URL):
    zip = str(zip_code)
    response = urllib.request.urlopen(API_URL + zip)
    xml_result = response.read()
    response.close()
    elem = xml.etree.ElementTree.fromstring(xml_result)
    long = elem.findtext(".//x")
    lat = elem.findtext(".//y")
    data = [zip_code, lat, long]
    return data

URL = 'http://geoapi.heartrails.com/api/xml?method=searchByPostal&postal='

for i in range(len(df)):
    lat_long.append(geo(df.Postcode[i], URL))

df_lat_long = pd.DataFrame(lat_long, columns=['Postcode', 'Latitude', 'Longitude'])  
df_lat_long.head()

,Postcode,Latitude,Longitude
0,4710052,35.104364,137.139678
1,4700335,35.123342,137.185037
2,4710873,35.063098,137.165974
3,4710835,35.05884,137.133589
4,4442832,35.244875,137.372781


In [7]:
df = pd.merge(df, df_lat_long, on='Postcode')
df.head()

,Postcode,AreaJP,Pref,Borough,Area,Population,Latitude,Longitude
0,4710052,逢妻町,AICHI KEN,TOYOTA SHI,AIZUMACHO,684,35.104364,137.139678
1,4700335,青木町,AICHI KEN,TOYOTA SHI,AOKICHO,"4,527",35.123342,137.185037
2,4710873,秋葉町,AICHI KEN,TOYOTA SHI,AKIBACHO,"1,906",35.063098,137.165974
3,4710835,曙町,AICHI KEN,TOYOTA SHI,AKEBONOCHO,"1,055",35.05884,137.133589
4,4442832,浅谷町,AICHI KEN,TOYOTA SHI,AZAKAICHO,157,35.244875,137.372781


In [8]:
df = df[['Postcode', 'Area', 'Population', 'Latitude', 'Longitude']]
df.head()

,Postcode,Area,Population,Latitude,Longitude
0,4710052,AIZUMACHO,684,35.104364,137.139678
1,4700335,AOKICHO,"4,527",35.123342,137.185037
2,4710873,AKIBACHO,"1,906",35.063098,137.165974
3,4710835,AKEBONOCHO,"1,055",35.05884,137.133589
4,4442832,AZAKAICHO,157,35.244875,137.372781


### Foursquare API

In [9]:
#Input ID, Secret, version, limit, query and radius
#CLIENT_ID = '' # your Foursquare ID
#CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
query = 'レストラン'
radius = 5000

In [11]:
results = requests.get(url).json()["response"]['venues']
results

[{'id': '4e6deeb12271a8cabffdf931',
  'name': 'レストラン東郷 トヨタ店',
  'location': {'address': '神田町1-7-1',
   'lat': 35.081473,
   'lng': 137.153612,
   'labeledLatLngs': [{'label': 'display',
     'lat': 35.081473,
     'lng': 137.153612}],
   'distance': 2335,
   'postalCode': '471-0860',
   'cc': 'JP',
   'city': '豊田市',
   'state': '愛知県',
   'country': '日本',
   'formattedAddress': ['神田町1-7-1', '豊田市, 愛知県', '471-0860', '日本']},
  'categories': [{'id': '4bf58dd8d48988d1c4941735',
    'name': 'Restaurant',
    'pluralName': 'Restaurants',
    'shortName': 'Restaurant',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1570445969',
  'hasPerk': False},
 {'id': '4bd3cbbe41b9ef3bea1601e6',
  'name': 'インドレストラン ナンハウス 豊田店',
  'location': {'address': '豊栄町6-121-3',
   'crossStreet': '国道248号',
   'lat': 35.04082476137397,
   'lng': 137.15351379381232,
   'labeledLatLngs': [{'label': 'display',
     'lat': 35.0

In [23]:
latitude = df.Latitude[2]
longitude = df.Longitude[2]
res_info = []
    
#set URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, query, radius, LIMIT)
    
#get results
results = requests.get(url).json()["response"]['venues']
#print(len(results))
for i in range(len(results)):
    name = results[i]['name']
    category = results[i]['categories'][0]['name']
    lat = results[i]['location']['lat']
    lng = results[i]['location']['lng']
    info = [name, category, lat, lng]
    res_info.append(info)
    print("{},{},lat:{}lng:{}".format(name, category, lat, lng))

レストラン東郷 トヨタ店,Restaurant,lat:35.081473lng:137.153612
インドレストラン ナンハウス 豊田店,Indian Restaurant,lat:35.04082476137397lng:137.15351379381232
豊田市役所 レストランひまわり,Restaurant,lat:35.08342lng:137.156378
Welcome Nepal Restaurant (ウェルカム ネパール レストラン),Asian Restaurant,lat:35.059215564198325lng:137.13019162416458
レストランバーLUCE,Restaurant,lat:35.085533lng:137.15552
レストラン ボナール,Restaurant,lat:35.086628lng:137.157533
レストラン ブルージュ,French Restaurant,lat:35.04160974354522lng:137.13689982891083
フォレスタヒルズ レストラン フォンターナ,French Restaurant,lat:35.04017127828179lng:137.1852597759574
タイレストラン タニサラ,Thai Restaurant,lat:35.085093lng:137.154576
ハンバーグレストラン キッチンフォレスト,Restaurant,lat:35.046371lng:137.140905
名鉄レストラン,Food Court,lat:35.033617lng:137.135467
フォレスタヒルズ レストラン和食 花乃里,Restaurant,lat:35.04043lng:137.185666
カフェレストランEL SUR,Café,lat:35.02175521850586lng:137.17401123046875
レストランヴェルデロッソ,Restaurant,lat:35.084150924934775lng:137.16967135572173
ほがらか T-FACE店,Japanese Restaurant,lat:35.08736123597804lng:137.15483508236815
Koyomi,Food Court

In [27]:
dname = ["name", "category", "Latitude", "Longitude"]
df_res = pd.DataFrame(res_info, columns=dname)
df_res.head()

,name,category,Latitude,Longitude
0,レストラン東郷 トヨタ店,Restaurant,35.081473,137.153612
1,インドレストラン ナンハウス 豊田店,Indian Restaurant,35.040825,137.153514
2,豊田市役所 レストランひまわり,Restaurant,35.083420,137.156378
3,Welcome Nepal Restaurant (ウェルカム ネパール レストラン),Asian Restaurant,35.059216,137.130192
4,レストランバーLUCE,Restaurant,35.085533,137.155520


In [28]:
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

In [31]:
address  =  'Toyota'
geolocator = Nominatim(user_agent="Toyota_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Toyota_map = folium.Map(location=[latitude, longitude], zoom_start=15)

for lat, lng, name, category in zip(df_res['Latitude'], df_res['Longitude'], df_res['name'], df_res['category']):
    label = '{}, {}'.format(name, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toyota_map)
    
Toyota_map